Gaussian process notebook

In [ ]:
import tempfile
from pathlib import Path

import numpy as np
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

from eddymotion import model
from eddymotion.data.dmri import DWI
from eddymotion.data.splitting import lovo_split

datadir = Path("../../test")  # Adapt to your local path or download to a temp location using wget

kernel = DotProduct() + WhiteKernel()

dwi = DWI.from_filename(datadir / "dwi.h5")

_dwi_data = dwi.dataobj
# Use a subset of the data for now to see that something is written to the
# output
# bvecs = dwi.gradients[:3, :].T
bvecs = dwi.gradients[:3, 10:15].T   # b0 values have already been masked
# bvals = dwi.gradients[3:, 10:13].T  # Only for inspection purposes: [[1005.], [1000.], [ 995.]]
dwi_data = _dwi_data[60:63, 60:64, 40:45, 10:15]

# ToDo
# Provide proper values/estimates for these
a = 1
h = 1  # should be a NIfTI image

num_iterations = 5
gp = model.GaussianProcessModel(
    dwi=dwi, a=a, h=h, kernel=kernel, num_iterations=num_iterations
)
indices = list(range(bvecs.shape[0]))
# ToDo
# This should be done within the GP model class
# Apply lovo strategy properly
# Vectorize and parallelize
result_mean = np.zeros_like(dwi_data)
result_stddev = np.zeros_like(dwi_data)

for idx in indices:
    lovo_idx = np.ones(len(indices), dtype=bool)
    lovo_idx[idx] = False
    X_train = bvecs[lovo_idx]
    X_test = bvecs[~lovo_idx]
    y_train = dwi_data[..., lovo_idx]
    y_test = dwi_data[..., ~lovo_idx]
    x, y, z, n_train  = y_train.shape
    y_train_flat = y_train.reshape(n_train, x*y*z )

    gp.fit(X_train, y_train_flat)

    # Predict for the test data
    pred_mean, pred_stddev = gp.predict(X_test, return_std=True)  # Can take multiple values X[:2, :]

    # Reshape the predictions back to the original voxel grid
    pred_mean_reshaped = pred_mean.reshape(dwi_data.shape[:-1])
    pred_stddev_reshaped = pred_stddev.reshape(dwi_data.shape[:-1])

    # Store the prediction result
    result_mean[..., idx] = pred_mean_reshaped
    result_stddev[..., idx] = pred_stddev_reshaped

Plot the data

In [ ]:
from matplotlib import pyplot as plt 
%matplotlib inline

s = dwi_data[1, 1, 2, :]
s_hat_mean = result_mean[1, 1, 2, :]
s_hat_stddev = result_stddev[1, 1, 2, :]
x = np.asarray(indices)

fig, ax = plt.subplots()
ax.plot(x, s_hat_mean, c="orange", label="predicted")
plt.fill_between(
    x.ravel(),
    s_hat_mean - 1.96 * s_hat_stddev,
    s_hat_mean + 1.96 * s_hat_stddev,
    alpha=0.5,
    color="orange",
    label=r"95% confidence interval",
)
plt.scatter(x, s, c="b", label="ground truth")
ax.set_xlabel("bvec indices")
ax.set_ylabel("signal")
ax.legend()
plt.title("Gaussian process regression on dataset")

plt.show()

Plot the DWI signal for a given voxel
Compute the DWI signal value wrt the b0 (how much larger/smaller is and add that delta to the unit sphere?) for each bvec direction and plot that?

In [ ]:
# from mpl_toolkits.mplot3d import Axes3D
# fig, ax = plt.subplots()
# ax = fig.add_subplot(111, projection='3d')
# plt.scatter(xx, yy, zz)

Plot the DWI signal brain data


In [ ]:
# plot_dwi(dmri_dataset.dataobj, dmri_dataset.affine, gradient=data_test[1], black_bg=True)

Plot the predicted DWI signal

In [ ]:
# plot_dwi(predicted, dmri_dataset.affine, gradient=data_test[1], black_bg=True);